# A new path: /echo

Now we're going to write a new path: `/echo`.

Our goal is the following:

- when invoking `/datetime/v1/echo` the API will return 
  the current datetime;
- the response is enveloped in the following example json object:

```
{ "timestamp": "2019-12-25T00:00:00Z" }
```

- the status code for a successful response is `200`

### Exercise: write /echo specs

Edit the [ex-04-02-echo-ok.yaml](/edit/notebooks/oas3/ex-04-02-echo.yaml) and write the `echo` specifications:

1- define the new `Datetime` schema to be used in the response;

2- define the new `/echo` path supporting the `get` method and:

  * always write proper `summary` and `description` fields
  
3- `get /echo` possible responses are:

  * `200` returning a `Datetime` in json format, with a complete `example` for mocks
  * `503` returning a `problem+json`
  

4- don't forget `operationId` !

Hint: feel free to reuse as much yaml code as possible.

### Exercise: test /echo mocks

Run your spec [in the terminal](/terminals/1) and check that it properly returns the mock objects.

Use 

```
connexion run --mock all /code/notebooks/oas3/ex-04-02-echo-ok.yaml
```

In [1]:
# Use this cell to test the output
!curl http://localhost:5000/datetime/v1/echo -vk

*   Trying 127.0.0.1...
* TCP_NODELAY set
* Connected to localhost (127.0.0.1) port 5000 (#0)





* HTTP 1.0, assume close after body






{
  "detail": "The requested URL was not found on the server. If you entered the URL manually please check your spelling and try again.",
  "status": 404,
  "title": "Not Found",
  "type": "about:blank"
}
* Curl_http_done: called premature == 0
* Closing connection 0


## Request parameters

OAS3 allows to specify parameters both in `path`, `query` and  `headers`.

Interoperable APIs should follow simple rules when using parameters:

- define parameter conventions for common patterns, eg: limit, offset;
- use standard HTTP headers when possible.

As many APIs implement pagination, it's a good think to converge to a common
set of parameters to use. Our choice is:

- limit: max number of entries to retrieve
- offset: the number of entries to skip in a paginated request
- cursor: an identifier (cursor) of the first entry to be returned  
  [Slack APIs provide an example of cursor-based pagination](https://api.slack.com/docs/pagination)

## Adding request parameters to `/echo`

`Query` and `header` request parameters can be defined in 

```
components:
  parameters:
    a_sample_parameter:
      ...
```

In [2]:
# Here's the `limit` parameter definition, including
# - it's a `query` parameter
# - its name
# - its schema
print(show_component('https://teamdigitale.github.io/openapi/0.0.5/definitions.yaml#/parameters/limit'))

description: How many items to return at one time (max 100)
in: query
name: limit
schema:
  format: int32
  type: integer



### Exercise: adding parameters to `/echo`

Let's add a `tz` parameter to `/echo`:

- create a `#/components/schemas/Timezone` schema definition. Valid example values are:

  * 'Europe/Rome'
  * 'UTC' (default)
  * 'Asia/Calcutta'
  
- create a `#/components/parameters/tz` parameter definition;
- add the `tz` query parameter to `get /echo` path checking
  the [official OAS 3.0.2 documentation](https://github.com/OAI/OpenAPI-Specification/blob/master/versions/3.0.2.md#examples)
- add a `400 Bad Request` response in case the timezone is not in the list

Finally, check that you can run the spec.

```
connexion run --mock all /code/notebooks/oas3/ex-04-02-echo-ok.yaml
```

In [3]:
# Use this cell to test your api

### Implement get_echo

Let's implement the `get_echo` in [api.py](/edit/notebooks/oas3/api.py) function that:

- takes the `tz` parameter, which defaults to `UTC`;
- returns a `Datetime` object in its current timezone.

In [4]:
# hint: get the timezone from 
!pip install pytz

In [12]:
import pytz 
tz = pytz.timezone('Europe/Rome')
print(tz.zone)

Europe/Rome


In [ ]:
# No timezones in Neverland :)
'Neverland' in pytz.all_timezones

In [ ]:
### Exercise solution

In [ ]:
def get_echo(tz='UTC'):
    if tz not in pytz.all_timezones:
        return problem(
            status=400,
            title="Bad Timezone",
            detail="The specified timezone is not valid",
            ext={"valid_timezones": pytz.all_timezones}
        )
    d = datetime.now(tz=pytz.timezone(tz))
    return {"timestamp": d.isoformat().replace('+00:00', 'Z')}
    

Now  [run the spec in a terminal](/terminals/1) using

```
cd /code/notebooks/oas3/
connexion run /code/notebooks/oas3/ex-04-02-echo-ok.yaml
```

In [ ]:
render_markdown(f'''
play a bit with the [Swagger UI]({api_server_url('ui')})

and try making a request!
''')

In [18]:
## TODO do we have enough time to show flask_testing?

# Check that default works
!curl http://localhost:5000/datetime/v1/echo -kv

*   Trying 127.0.0.1...
* TCP_NODELAY set
* Connected to localhost (127.0.0.1) port 5000 (#0)





* HTTP 1.0, assume close after body






{
  "timestamp": "2019-07-05T09:56:00.357602Z"
}
* Curl_http_done: called premature == 0
* Closing connection 0


In [17]:
# Test a valid timezone
!curl http://localhost:5000/datetime/v1/echo?tz=Europe/Rome -kv

*   Trying 127.0.0.1...
* TCP_NODELAY set
* Connected to localhost (127.0.0.1) port 5000 (#0)
> GET /datetime/v1/echo?tz=Europe/Rome HTTP/1.1
> Host: localhost:5000
> User-Agent: curl/7.52.1
> Accept: */*
> 
* HTTP 1.0, assume close after body
< HTTP/1.0 200 OK
< Content-Type: application/problem+json
< Content-Length: 54
< Server: Werkzeug/0.15.4 Python/3.6.6
< Date: Fri, 05 Jul 2019 09:55:53 GMT
< 
{
  "timestamp": "2019-07-05T11:55:53.986946+02:00"
}
* Curl_http_done: called premature == 0
* Closing connection 0


In [19]:
# Test an invalid timezone
!curl http://localhost:5000/datetime/v1/echo?tz=Frittole -kv

*   Trying 127.0.0.1...
* TCP_NODELAY set
* Connected to localhost (127.0.0.1) port 5000 (#0)





* HTTP 1.0, assume close after body






{
  "detail": "The specified timezone is not valid",
  "status": 400,
  "title": "Bad Timezone",
  "type": "about:blank",
  "valid_timezones": [
    "Africa/Abidjan",
    "Africa/Accra",
    "Africa/Addis_Ababa",
    "Africa/Algiers",
    "Africa/Asmara",
    "Africa/Asmera",
    "Africa/Bamako",
    "Africa/Bangui",
    "Africa/Banjul",
    "Africa/Bissau",
    "Africa/Blantyre",
    "Africa/Brazzaville",
    "Africa/Bujumbura",
    "Africa/Cairo",
    "Africa/Casablanca",
    "Africa/Ceuta",
    "Africa/Conakry",
    "Africa/Dakar",
    "Africa/Dar_es_Salaam",
    "Africa/Djibouti",
    "Africa/Douala",
    "Africa/El_Aaiun",
    "Africa/Freetown",
    "Africa/Gaborone",
    "Africa/Harare",
    "Africa/Johannesburg",
    "Africa/Juba",
    "Africa/Kampala",
    "Africa/Khartoum",
    "Africa/Kigali",
    "Africa/Kinshasa",
    "Africa/Lagos",
 